In [8]:
# by Joh Schöneberg 2018

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from numpy import genfromtxt


import pandas as pd

from mpl_toolkits import mplot3d
#%matplotlib notebook

import pylattice

import pylattice.functions.InputProcessing as inproc
import pylattice.functions.TiffProcessing as tiffproc


import sys
sys.path.append("/Users/johannesschoeneberg/git/JohSchoeneberg/pyLattice/src/python/classes/")
#import pylattice.functions.InterfaceChimeraX as icx
import TrackUnprocessed as trackUnprocessed




# input data

In [2]:
path = '/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/'


filepath = path+'_inputParameters.csv'
print(filepath)
inputParameters = pd.read_csv(filepath,names=['key','value'])


inputDataFolder = inproc.getInputParameter(inputParameters,"inputDataFolder")
outputDataFolder = inproc.getInputParameter(inputParameters,"outputDataFolder")
trackingCsvFilename = inproc.getInputParameter(inputParameters,"trackingCsvFilename")


slave_outputDataFolder  = inproc.getInputParameter(inputParameters,"slave_outputDataFolder")
master_outputDataFolder  = inproc.getInputParameter(inputParameters,"master_outputDataFolder")

slave_uniqueFilenameString = inproc.getInputParameter(inputParameters,"slave_uniqueFilenameString")
master_uniqueFilenameString = inproc.getInputParameter(inputParameters,"master_uniqueFilenameString")


slave_trackingProcessedBildFilename = inproc.getInputParameter(inputParameters,"slave_trackingProcessedBildFilename")
master_trackingProcessedBildFilename = inproc.getInputParameter(inputParameters,"master_trackingProcessedBildFilename")



framerate_msec = float(inproc.getInputParameter(inputParameters,"framerate_msec"))
movieLength = int(inproc.getInputParameter(inputParameters,"movieLength"))




/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/_inputParameters.csv


In [3]:
filepath = path +outputDataFolder+'/'+trackingCsvFilename
print(filepath)
data = pd.read_csv(filepath,header=0)
data.columns = ["trackId", "tracklength", "frameId", "particleId", "x", "y", "z", "A", "noIdea1", "noIdea2", "noIdea3", "noIdea4"]

dataUnique = data.drop_duplicates(subset='trackId')

lengthTreshold = 5 #frames
trackIdsLong = (data[data['tracklength'] > lengthTreshold]).drop_duplicates(subset='trackId')['trackId'].values
longTracks = data[data['tracklength'] > lengthTreshold]
longTracks

print("total number of tracks: {}\nunique tracks: {}\nlongtracks (len>{}) {}".format(len(data),len(dataUnique),lengthTreshold,len(longTracks.drop_duplicates(subset='trackId'))))

/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackedFeatures.csv
total number of tracks: 1728
unique tracks: 484
longtracks (len>5) 94


# draw every track in one file

In [4]:
longtracks = data[data['tracklength'] >= 5]
print(len(longtracks))
print(len(np.unique(data['trackId'].values)))
print(np.unique(data['trackId'].values))

1103
484
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 2

In [5]:
#plot them all
import math

data = data

drawColor = 'red'

output_filename = 'tracksUnprocessed.bild'
filepath = path+outputDataFolder+'/'+output_filename
print(filepath)
file = open(filepath,'w') 

file.write(".transparency 0.5\n")
file.write(".color "+drawColor+"\n")

for trackID in np.unique(data['trackId'].values):
    
    line = ".comment trackID"+str(trackID)+"\n"
    file.write(line)
    
    
    track = data[data['trackId'] == trackID]
    tracklength = track['tracklength'].iloc[0]
    
    for i in range(1,tracklength):
        tzero = track.iloc[i-1]
        tone = track.iloc[i]
        

        # Data for a three-dimensional line
        x0 = float(tzero['x'])
        y0 = float(tzero['y'])
        z0 = float(tzero['z'])
        A0 = float(tzero['A'])
        
        x1 = float(tone['x'])
        y1 = float(tone['y'])
        z1 = float(tone['z'])
        A1 = float(tone['A'])
        
        if(math.isnan(x0) or math.isnan(y0) or math.isnan(z0) or math.isnan(x1) or math.isnan(y1) or math.isnan(z1)):
            line = ".arrow "+str(x0)+" "+str(y0)+" "+str(z0)+" "+str(x1)+" "+str(y1)+" "+str(z1)+"\n" #" "+str(radius)+"\n"        
            print(line)
            continue
        
        
        #.arrow x1 y1 z1 x2 y2 z2 [r1 [r2 [rho]]] 
        
        line = ".arrow "+str(x0)+" "+str(y0)+" "+str(z0)+" "+str(x1)+" "+str(y1)+" "+str(z1)+"\n" #" "+str(radius)+"\n"        
        file.write(line)
        
file.close()

/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_all.bild
.arrow 93.3853 17.88712 9.318319 nan nan nan

.arrow nan nan nan nan nan nan

.arrow 10.72135 37.31103 47.03282 nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan nan nan nan

.arrow 26.86486 14.14842 58.80415 nan nan nan

.arrow 82.29193 43.89157 46.4369 nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan nan nan nan

.arrow 42.58184 50.79379 64.49817 nan nan nan

.arrow nan nan nan nan nan nan

.arrow 2.899544 59.75309 56.93139 nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan nan nan nan



# create a .bild file for every track individually

In [6]:
import timeit


trackIdListInvestigated = trackIdsLong

start_time = timeit.default_timer()

tracks = []
cm0 = []
len0 = []
Amean0 = []
maxDist0 = []



for i in range(0,len(trackIdListInvestigated)):
    if(i %1000 == 0):
        print(str(i)+"/"+str(len(trackIdListInvestigated)))
    a = trackUnprocessed.TrackUnprocessed(data[data['trackId'] == trackIdListInvestigated[i]])
    tracks.append(a)
    cm0.append(a.cm)
    len0.append(a.len)
    Amean0.append(a.Amean)
    maxDist0.append(a.maxDist)
    
elapsed = timeit.default_timer() - start_time
print('time elapse: '+str(np.round(elapsed,decimals=2))+'s')

0/94
time elapse: 0.22s


In [7]:
for track in tracks:
    numberOfZeros = int(np.ceil(np.log10(movieLength+1)))
    output_filename = 'trackUnprocessed_'+str(track.id).zfill(numberOfZeros)+'.bild'
    filepath = path+outputDataFolder+'/'+output_filename
    print(filepath)
    track.writeBILD(filepath,color='red')

/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_01.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_02.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_03.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_04.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_05.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_06.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_07.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_08.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocessed_09.bild
/Users/johannesschoeneberg/Dropbox/pylattice_testData/imaging_data/./output/trackUnprocesse